In [1]:
import torch
import warnings

device = 'cpu'
torch.set_default_device(device)

warnings.filterwarnings("ignore")

In [2]:
! rm -r lightning_logs/*

In [3]:
from my_dataset import create_dataset

In [4]:
from torch.utils.data import Dataset

class SimpleData(Dataset):
    def __init__(self, m):
        self.m = m
        self.X, self.y = create_dataset(m,True)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return self.m

In [5]:
import lightning as L
import torch

class OneLayerNetwork(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.weights = torch.nn.Linear(3,1)
        self.lr = 2e-3

    def forward(self,X):
        return self.weights(X)

    def training_step(self,batch):
        X,y = batch
        y_pred = self.forward(X)
        loss = torch.nn.functional.mse_loss(y_pred,y)
        self.log("train_loss", loss)
        self.log("parameters - w_a", self.weights.weight[0][0])
        self.log("parameters - w_b", self.weights.weight[0][1])
        self.log("parameters - w_c", self.weights.weight[0][2])
        self.log("parameters - bias", self.weights.bias[0])
        return loss


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr=self.lr)
        return optimizer

In [6]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

data = SimpleData(10000)

train,test = train_test_split(data, test_size=0.15)

train_loader = DataLoader(train)
test_loader = DataLoader(test)

model = OneLayerNetwork()


In [7]:
trainer = L.Trainer(limit_train_batches=100, max_epochs=50)

trainer.fit(model=model, train_dataloaders=train_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type   | Params | Mode 
-------------------------------------------
0 | weights | Linear | 4      | train
-------------------------------------------
4         Trainable params
0         Non-trainable params
4         Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|█████████████████████| 100/100 [00:00<00:00, 218.49it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█████████████████████| 100/100 [00:00<00:00, 216.56it/s, v_num=0]


In [8]:
model.weights.weight

Parameter containing:
tensor([[ 2.0000e+00, -3.9219e-08, -5.0000e-01]], requires_grad=True)

In [9]:
model.weights.bias

Parameter containing:
tensor([4.0345e-07], requires_grad=True)

In [10]:
model(torch.tensor([-124,451,3],dtype=torch.float32))

tensor([-249.4997], grad_fn=<ViewBackward0>)